# Tensorflow Basics 

#### Basics
* Importing library
* Testing devices

In [2]:
import numpy as np
import tensorflow as tf
from pprint import pprint

print(f'Tensorflow Version : {tf.__version__}')

if tf.config.list_physical_devices('GPU'):
    print("GPU detected")

Tensorflow Version : 2.6.0
GPU detected


### Array creation using Numpy and Tensorflow

In [3]:
np_x = np.ones((3,3))
tf_x = tf.ones((3,3))

#### Insights in a nutshell 
* Tensors are immutable.
* TensorFlow automatically converts Numpy array to Tensor type.
* But, if the operation changes between devices, example : CPU or GPU, a copy is made and then the follwing happens.

In [4]:
var1 = tf.Variable(initial_value=np_x) #example passed np is automatically consumed and converted to TF tensor

#### Computing Gradients

In [5]:
input_ = tf.Variable(tf.ones((3,3),dtype=tf.float32)*3)
with tf.GradientTape() as t:
    ops = input_*3
gradient = t.gradient(ops,input_)

pprint(f"Gradient : {gradient}")

'Gradient : [[3. 3. 3.]\n [3. 3. 3.]\n [3. 3. 3.]]'


#### Creating custom Layer
#### Keypoints to note when playing with custom layers:

* Method of creating weights (Trainable and Non-Trainable variables).
* Updating get_config method to make the layer serializable.
* * The below method is better if incoming input shape is unknown.
* * Use build method to construct weights or any variables and call method for computation.

In [6]:
class SimpleLayer(tf.keras.layers.Layer):
    def __init__(self,units,name):
        self.units = units
        super(SimpleLayer,self).__init__(name=name)
        
    def build(self,input_shape):
        self.w = self.add_weight(shape=(input_shape[-1],self.units),initializer="random_normal",trainable=True)
        self.b = self.add_weight(shape=(self.units,),initializer="random_normal",trainable=True)
        
    def call(self,inputs):
        return tf.matmul(inputs, self.w) + self.b
    
    def get_config(self):
        config = super(SimpleLayer, self).get_config()
        config.update({"units":self.units})
        return config

#### Execution of layers

In [7]:
layer = SimpleLayer(units=10,name="layer1")
input_matrix = tf.ones(shape=(2,4))
output_matrix = layer(input_matrix)
print(f"input_matrix shape : {input_matrix.shape}")
print(f"output_matrix shape : {output_matrix.shape}")

input_matrix shape : (2, 4)
output_matrix shape : (2, 10)


#### Creating simple mask generating and consuming layers.

In [9]:
class Custom_mask_gen_layer(tf.keras.layers.Layer):
    def __init__(self,neurons,**kwargs):
        super(Custom_mask_gen_layer,self).__init__(**kwargs)
        
        self.dense_layer = layers.Dense(neurons)
        
    def call(self,x):
        return self.dense_layer(x)
    
    
    def compute_mask(self,x,mask):
        return x!=2
    
    
class Custom_mask_consume_layer(tf.keras.layers.Layer):
    def __init__(self,neurons,**kwargs):
        super(Custom_mask_consume_layer,self).__init__(**kwargs)
        
        self.dense_layer = layers.Dense(neurons)
        self.supports_masking = True
        
    def call(self,x,mask=None):
        if mask is not None:
            print(tf.cast(mask,tf.int32))
        return self.dense_layer(x)

#### Demo on chaining layers to encapsulate single model

In [ ]:
Input_Node1 = layers.Input(shape=(8,))
layer1 = Custom_mask_gen_layer(8)(Input_Node1)
layer2 = Custom_mask_consume_layer(8)(layer1)
child1 = tf.keras.Model(Input_Node1,layer2)

Input_Node2 = layers.Input(shape=(8,))
Input_Node2_2 = layers.Input(shape=(8,))
x1 = Custom_mask_gen_layer(8)(Input_Node2)
x2 = Custom_mask_gen_layer(8)(Input_Node2_2)
x = x1+x2
x = Custom_mask_consume_layer(8)(x)
child2 = tf.keras.Model((Input_Node2,Input_Node2_2),x)

child2_out = child2((Input_Node2,layer2))
CHAIN_MODEL = tf.keras.Model((Input_Node1,Input_Node2),child2_out)